In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
from tqdm import tqdm
import pandas as pd
import random
import ast
from collections import defaultdict

# Input data files are available in the read-only                       "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/assignment-1-nlp/train.csv') # loading training data
data = []
for index, row in tqdm(df.iterrows()):
    data.append(ast.literal_eval(row['tagged_sentence'])) # changing data-type of entries from 'str' to 'list'

In [ ]:
df = pd.read_csv('/kaggle/input/assignment-1-nlp/test_small.csv') # loading test data
test_data = {} 
for index, row in tqdm(df.iterrows()):
    test_data[row['id']] = ast.literal_eval(row['untagged_sentence']) # changing data-type of entries from 'str' to 'list'

In [ ]:
def display_data(sentence_index):
    '''
        Input : 'sentence_index' (int) -> index of a sentence in training data
        Output: None
    '''
    sentence = data[sentence_index]
    print("TOKEN -> TAG")
    print('...')
    for token, tag in sentence:
        print(token, '>', tag)
sentence_index = random.choice(range(len(data)))
display_data(sentence_index)

In [ ]:
# cell to show the frequency of each distinct (slack or native) present in the training data
from collections import Counter
distinct_tags = []
word_tags = []
def store_tags():
    
    global distinct_tags
    global word_tags
    
    for sent in data:
        word_tags.append(('START','START'))
        for words, tag in sent:
            word_tags.extend([(tag, words)])
        word_tags.append(('END','END'))
    
store_tags()
tags=[]
for tag, words in word_tags:
    tags.append(tag)
distinct_tags=list(set(tags))
count_tags = {}
for tag, count in Counter(tags).items():
    count_tags[tag] = count

In [ ]:
tag_list = [tag for sentence in data for word,tag in sentence ]
print(len(data))
print((tag_list[:10]))

In [ ]:
unique_tags = {tag for sentence in data for word,tag in sentence}
print(unique_tags)
print(len(unique_tags))

In [ ]:
tag_counts = {}   
for sentence in data:
    for word , tag in sentence:
        tag_counts[tag] = tag_counts.get(tag, 0) + 1
print(tag_counts)

In [ ]:
default_prob = 1e-10

# **HMM Model**

In [ ]:
def calculate_start_probabilities(corpus):
    start_prob = {}
    for tag in unique_tags:
        start_prob[tag] = default_prob
    for sentence in corpus:
        if sentence:  # Check if the sentence is not empty
            start_prob[sentence[0][1]] += 1
    total_sentences = sum(start_prob.values())
    start_prob = {state: count / total_sentences for state, count in start_prob.items()}
    return (start_prob)

In [ ]:
len(calculate_start_probabilities(data))

In [ ]:
#computing transition probability
def transition_proba(training_corpus):
    transition = {}
    for sentence in training_corpus:
        for index in range(0, len(sentence) - 1):
            current = sentence[index][1]
            next_elem = sentence[index + 1][1]
            if(current not in transition):
                transition[current] = {}
            if(next_elem not in transition[current]):
                transition[current][next_elem] = 0
                
            transition[current][next_elem] += 1
            
    for curr_state in transition:
        total_transitions = sum(transition[curr_state].values())
        for next_state,count in transition[curr_state].items():
            transition[curr_state][next_state] = count / total_transitions
            
    return transition

In [ ]:
len(transition_proba(data))

In [ ]:
def calculate_emission_probabilities(corpus):
    
    emission_prob = {}
    for sentence in corpus:
        for word, state in sentence:
            if(state not in emission_prob):
                emission_prob[state] = {}
            if(word not in emission_prob[state]):
                emission_prob[state][word] = 0
    
            emission_prob[state][word] += 1

    for state in emission_prob:
        total_emissions = sum(emission_prob[state].values())
        emission_prob[state] = {word: count / total_emissions
                                for word, count in emission_prob[state].items()}
        
    return (emission_prob)

In [ ]:
something = (calculate_emission_probabilities(data))

In [ ]:
states = list(unique_tags)
start_prob = calculate_start_probabilities(data)
transition_probs = transition_proba(data)
emission_probs = calculate_emission_probabilities(data)

In [ ]:
det = ['a','A','an','the','The','An']

# **Viterbi Algorithm**

In [ ]:
def viterbi_algorithm(sentence):
    n = len(sentence)
    m = len(states)
    
    dp = np.zeros((n, m))
    backpointer = np.zeros((n, m), dtype=int)
    
    # Initialization step
    for i in range(m):
        word = sentence[0]
        dp[0][i] = start_prob.get(states[i], default_prob) * emission_probs[states[i]].get(word, default_prob)
    
    # Recursion step
    for t in range(1, n):
        for j in range(m):
            word = sentence[t]
            max_prob = 0
            max_index = 0

            for i in range(m):
                transition_prob_ij = transition_probs.get(states[i], default_prob).get(states[j], default_prob)
                prob = dp[t-1][i] * transition_prob_ij * emission_probs[states[j]].get(word, default_prob)

                if prob > max_prob:
                    max_prob = prob
                    max_index = i
            
            dp[t][j] = max_prob
            backpointer[t][j] = max_index

    # Backtrack to find the best sequence of tags
    best_sequence = [0] * n
    best_index = np.argmax(dp[n-1, :])

    for t in range(n-1, -1, -1):
        best_sequence[t] = states[best_index]
        best_index = backpointer[t][best_index]

    return best_sequence

In [ ]:
def hmm_tagger_util(sent_id,untagged_sentence):
    predicted_tags = viterbi_algorithm(untagged_sentence)
    tagged_sentence = []
    for index ,word in enumerate(list(untagged_sentence)):
        tagged_sentence.append((word, predicted_tags[index]))
    store_submission(sent_id, tagged_sentence)

In [ ]:
submission = {'id': [], 'tagged_sentence' : []} # dictionary to store tag predictions
# NOTE ---> ensure that tagged_sentence's corresponing 'id' is same as 'id' of corresponding 'untagged_sentence' in training data
def store_submission(sent_id, tagged_sentence):
    
    global submission
    submission['id'].append(sent_id)
    submission['tagged_sentence'].append(tagged_sentence)
    
def clear_submission():
    global submission
    submission = {'id': [], 'tagged_sentence' : []}

In [ ]:
for sent_id in tqdm(list(test_data.keys())):
    sent = test_data[sent_id]
    hmm_tagger_util(sent_id, sent)

In [ ]:
path_to_directory = '/kaggle/working/'
pd.DataFrame(submission).to_csv(path_to_directory +' my_hmm_submission.csv', index = False)

# **MEMM MODEL**

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
unique_tags = {tag for sentence in data for word,tag in sentence}
tag_dict = {tag: i for i, tag in enumerate(set(unique_tags))}
from_num_to_tag = {}
for key,value in tag_dict.items():
    from_num_to_tag[value] = key

In [ ]:
#Extracting Features

feature_vector = []
def feature(sentence):
        
    sentence_features = []
    for i, (word, tag) in enumerate(sentence):
        # Get the previous word if it exists, otherwise use None
        prev_word = sentence[i-1][0] if i > 0 else None

        # Extract features for the current word
        word_features = {
            'capitalize': int(word[0].isupper()),
            'digit_first': int(word[0].isdigit()),
            's_dig__e_alpha': int(word[0].isdigit() and word[-1].isalpha()),
            'p_anti': int(word.startswith('anti')),
            'p_pre': int(word.startswith('pre')),
            'p_un': int(word.startswith('un')),
            'p_dis': int(word.startswith('dis')),
            'p_inter': int(word.startswith('inter')),
            'p_mis': int(word.startswith('mis')),
            'p_non': int(word.startswith('non')),
            'p_over_under': int(word.startswith('over') or word.startswith('under')),
            'p_in_im': int(word.startswith('in') or word.startswith('im')),
            'p_en_em': int(word.startswith('en') or word.startswith('em')),
            's_able': int(word.endswith('able')),
            's_al_ial': int(word.endswith('al') or word.endswith('ial')),
            's_ed_ing': int(word.endswith('ed') or word.endswith('ing')),
            's_tion_ion': int(word.endswith('tion') or word.endswith('ion')),
            's_est': int(word.endswith('est')),
            's_less': int(word.endswith('less')),
            's_e_es': int(word.endswith('e') or word.endswith('es')),
            's_en': int(word.endswith('en')),
            's_ly': int(word.endswith('ly')),
            's_er': int(word.endswith('er')),
            's_\'s_s\'': int(word.endswith('\'s') or word.endswith('s\'')),
            'prev_word_det': int(prev_word in ('a', 'an', 'the') if prev_word else 0),
            'prev_word_det': int(prev_word in stop_words if prev_word else 0),

        }

        temp = []
        for key,value in word_features.items():
            temp.append(value);

        # Add the feature dictionary to the list of features for this sentence
        sentence_features.append(temp)

    return sentence_features


word_vector = []
tag_vector = []
Y_train = []

for sent in data:
    sent_feature = feature(sent)
    for l in sent_feature:
        feature_vector.append(l)
        
for sentence in data:
    for word,tag in sentence:
        word_vector.append(word)
        Y_train.append(tag_dict[tag])

In [ ]:
print(word_vector[:3])
print(Y_train[:3])
print(len(feature_vector))

# **Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing
le = preprocessing.LabelEncoder()

Y = np.array(Y_train)
softmax_2 = LogisticRegression( multi_class = 'multinomial', solver = 'lbfgs',max_iter = 100,penalty = 'l2')
softmax_2.fit(feature_vector,Y)

In [ ]:
feature_vector = np.array(feature_vector)
softmax_2.predict_proba(feature_vector[0].reshape(1,-1))

In [ ]:
def viterbi_algorithm_memm(sentence):
    n = len(sentence)
    m = len(states)
    
    dp = np.zeros((n, m))
    backpointer = np.zeros((n, m), dtype=int)
    
    
    # Initialization step
    to_pass = [(word,'_') for word in sentence]
    word_feature = feature(to_pass)
    temp = word_feature[0]
    temp = np.array(temp)
    
    probs_start = softmax_2.predict_proba(temp.reshape(1,-1))
    
    for i in range(m):
        dp[0][i] = probs_start[0][i]
        
    # Recursion step
    for t in range(1, n):
        temp = word_feature[t]
        temp = np.array(temp)
        probs_new = softmax_2.predict_proba(temp.reshape(1,-1))
        for j in range(m):
            word = sentence[t]
            max_prob = 0
            max_index = 0
            
            for i in range(m):
                prob = dp[t-1][i] * probs_new[0][j]

                if prob > max_prob:
                    max_prob = prob
                    max_index = i
            
            dp[t][j] = max_prob
            backpointer[t][j] = max_index

    # Backtrack to find the best sequence of tags
    best_sequence = [0] * n
    best_index = np.argmax(dp[n-1, :])

    for t in range(n-1, -1, -1):
        best_sequence[t] = states[best_index]
        best_index = backpointer[t][best_index]

    return best_sequence

In [ ]:
def memm_tagger_util(sent_id, untagged_sentence):
#     tagged_sentence = []
#     predicted_tag = []
#     to_pass = [(word,'_') for word in untagged_sentence]
#     word_feature = feature(to_pass)
#     for temp in word_feature:
#         temp = np.array(temp)
#         temp = temp.reshape(1,-1)
        
#         predicted = softmax_2.predict(temp)
#         tag = from_num_to_tag[predicted[0]]
#         predicted_tag.append(tag)
        
#     for index ,word in enumerate(list(untagged_sentence)):
#         tagged_sentence.append((word, predicted_tag[index])) 
    
#     print(tagged_sentence)
    predicted_tags = viterbi_algorithm_memm(untagged_sentence)
    tagged_sentence = []
    for index ,word in enumerate(list(untagged_sentence)):
        tagged_sentence.append((word, predicted_tags[index])) 
#     print(tagged_sentence)
    store_submission(sent_id, tagged_sentence)

In [ ]:
for sent_id in tqdm(list(test_data.keys())):
    sent = test_data[sent_id]
    memm_tagger_util(sent_id, sent)

In [ ]:
path_to_directory = '/kaggle/working/'
pd.DataFrame(submission).to_csv(path_to_directory +' wwww_submission.csv', index = False)